# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
weather_df = pd.read_csv (r'city_data.csv')
weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Bengkulu,93,ID,1644376239,75,-3.8004,102.2655,27.46,3.66
1,Port Lincoln,98,AU,1644376239,57,-34.7333,135.8667,21.50,5.16
2,Luba,77,GQ,1644376239,85,3.4568,8.5547,26.34,3.04
3,Beïnamar,63,TD,1644376240,16,8.6698,15.3813,21.19,1.95
4,Bredasdorp,0,ZA,1644376240,86,-34.5322,20.0403,15.71,2.77
...,...,...,...,...,...,...,...,...,...
607,Gusau,91,NG,1644376419,12,12.1628,6.6614,20.15,5.29
608,Ust-Kuyga,91,RU,1644376420,99,70.0167,135.6000,-39.16,3.99
609,Dhāmnod,99,IN,1644376420,48,22.2167,75.4667,20.23,1.40
610,Stromness,100,GB,1644376420,73,58.9650,-3.2960,3.98,17.85


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_df["Humidity"]

In [19]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
# Add layer
# Display figure
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
temp_file = weather_df.loc[(weather_df['Max Temp']>=60)&(weather_df['Max Temp']<=80)]
wind_file = temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather = wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
# Create a hotel_df
# Add a "Hotel Name" column to the DataFrame.
# Display the result

hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
 
    params["location"] = f"{lat},{lng}"
    
   
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
   
    hotel_name = requests.get(base_url, params=params)
    
   
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df 

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))